<a href="https://colab.research.google.com/github/asupraja3/ml-ng-notebooks/blob/main/LogisticRegression_PracticeLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Practice Lab: Logistic Regression

This lab will help you **practice logistic regression** concepts from Andrew Ng's Machine Learning course.

### You will practice:
1. Sigmoid function implementation and visualization  
2. Logistic regression hypothesis and cost function  
3. Gradient descent updates  
4. Decision boundary visualization  
5. Model evaluation (accuracy)  
6. Feature scaling and vectorization  
7. Regularization (overview + implementation exercise)



## 1. Sigmoid Function

The logistic regression hypothesis uses the **sigmoid function**:  
$ \sigma(z) = \frac{1}{1 + e^{-z}} $

✅ **Exercise:** Implement `sigmoid(z)` and test on values [-10, 0, 10].



## 2. Logistic Regression Hypothesis & Cost

**Hypothesis:**  
$ h(x) = \sigma(w^T x + b) $

**Loss for one example:**  
$ L = - \big( y \log(h(x)) + (1-y)\log(1-h(x)) \big) $

**Cost function:**  
$ J(w,b) = -\frac{1}{m} \sum_{i=1}^m \Big[ y^{(i)} \log(h(x^{(i)})) + (1-y^{(i)}) \log(1-h(x^{(i)})) \Big] $

✅ **Exercise:** Implement the cost function `compute_cost_logistic(X, y, w, b)`.



## 3. Gradient Descent for Logistic Regression

**Gradients:**

$ \frac{\partial J}{\partial w} = \frac{1}{m} \sum_{i=1}^m \big( h(x^{(i)}) - y^{(i)} \big) x^{(i)} $

$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m \big( h(x^{(i)}) - y^{(i)} \big) $


✅ **Exercise:** Implement `compute_gradient_logistic` and `gradient_descent_logistic`.



## 4. Decision Boundary Visualization

For 2D data, the decision boundary is defined by:  

$ w^T x + b = 0 $  

Expanding for two features $x_1$ and $x_2$:  

$ w_1 x_1 + w_2 x_2 + b = 0 $  

We can solve for $x_2$:  

$ x_2 = -\frac{w_1}{w_2} x_1 - \frac{b}{w_2} $


✅ **Exercise:** Train logistic regression on synthetic data and plot boundary.



## 5. Model Evaluation

$ \text{Accuracy} = \dfrac{\text{Number of correct predictions}}{\text{Total number of predictions}} $


✅ **Exercise:** Implement accuracy function.



## 6. Feature Scaling & Vectorization

- Scaling helps convergence.  
- Vectorization speeds up training.

✅ **Exercise:** Scale X and retrain.



## 7. Regularization (Bonus)

**Regularized Cost Function:**  

$ J(w,b) = \text{Logistic Loss} + \dfrac{\lambda}{2m} \sum_{j=1}^{n} w_j^2 $  

where the logistic loss is:  

$ -\dfrac{1}{m} \sum_{i=1}^m \Big[ y^{(i)} \log(h(x^{(i)})) + (1-y^{(i)}) \log(1-h(x^{(i)})) \Big] $


✅ **Exercise:** Modify cost and gradient functions to include L2 penalty.



## ✅ Takeaways

- Logistic regression outputs **probabilities** using sigmoid.  
- Cost function: **log-loss**, convex and well-suited for classification.  
- Gradient descent optimizes weights iteratively.  
- Decision boundary is linear in features.  
- Feature mapping + regularization improve performance.  
- Scaling accelerates convergence.

Now, experiment with different datasets, learning rates, and λ values! 🚀
